<a href="https://colab.research.google.com/github/adithyaac/Captcha-breaker/blob/main/Task%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files

uploaded = files.upload()  # Manually select your ZIP file


Saving captcha_dataset.zip to captcha_dataset.zip


In [5]:
!unzip {"captcha_dataset.zip"} -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/captcha_dataset/hard/1900_CONTrOL.png  
  inflating: /content/captcha_dataset/hard/1901_ConcERT.png  
  inflating: /content/captcha_dataset/hard/1902_ENHaNCE.png  
  inflating: /content/captcha_dataset/hard/1903_ExciTED.png  
  inflating: /content/captcha_dataset/hard/1904_JOurnEY.png  
  inflating: /content/captcha_dataset/hard/1905_HEaLtHy.png  
  inflating: /content/captcha_dataset/hard/1906_BrigHT.png  
  inflating: /content/captcha_dataset/hard/1907_feELinG.png  
  inflating: /content/captcha_dataset/hard/1908_coNcEpT.png  
  inflating: /content/captcha_dataset/hard/1909_iSOlaTe.png  
  inflating: /content/captcha_dataset/hard/190_HUNgry.png  
  inflating: /content/captcha_dataset/hard/1910_AcCeSs.png  
  inflating: /content/captcha_dataset/hard/1911_elEMENT.png  
  inflating: /content/captcha_dataset/hard/1912_cirClE.png  
  inflating: /content/captcha_dataset/hard/1913_CAmeRA.png  
  inflating: /content/cap